In [1]:
import functools as ft
import json
import os
import pandas as pd
import yaml

from pathlib import Path

In [2]:
main_dir = Path('../data/fixed/')
target_dir = Path('../data/dataset/')

In [3]:
# get all subfolders containing bags
bag_paths = [folder for folder in main_dir.rglob('**/') if any(subfolder in folder.__str__() for subfolder in ('T1\\', 'T2\\'))]
labels = {}

for bag_path in bag_paths:
    # split path into chunks
    subfolders = os.path.normpath(bag_path).split(os.sep)
    key = subfolders[-1]
    
    # get cmd_vel, imu_data, odom, and servo_data
    bag_files = [bag_path.joinpath(bag_file) for bag_file in os.listdir(bag_path) if bag_file in ('cmd_vel.csv', 'imu-data.csv', 'odom.csv', 'Servo_data.csv')]

    # read and merge tables
    dataframes = [pd.read_csv(bag_file) for bag_file in bag_files]
    dataframe = ft.reduce(lambda left, right: pd.merge(left, right, how='outer', on='Time'), dataframes)

    # clean resulting dataframe
    dataframe = dataframe[dataframe.columns.drop(list(dataframe.filter(regex='header')))]
    dataframe.columns = ['Time', 'linear.x', 'linear.y', 'linear.z', 'angular.x', 'angular.y', 'angular.z',
                         'orientation.x', 'orientation.y', 'orientation.z',
                         'orientation.w', 'orientation_covariance_0', 'orientation_covariance_1',
                         'orientation_covariance_2', 'orientation_covariance_3',
                         'orientation_covariance_4', 'orientation_covariance_5',
                         'orientation_covariance_6', 'orientation_covariance_7',
                         'orientation_covariance_8', 'angular_velocity.x', 'angular_velocity.y',
                         'angular_velocity.z', 'angular_velocity_covariance_0',
                         'angular_velocity_covariance_1', 'angular_velocity_covariance_2',
                         'angular_velocity_covariance_3', 'angular_velocity_covariance_4',
                         'angular_velocity_covariance_5', 'angular_velocity_covariance_6',
                         'angular_velocity_covariance_7', 'angular_velocity_covariance_8',
                         'linear_acceleration.x', 'linear_acceleration.y',
                         'linear_acceleration.z', 'linear_acceleration_covariance_0',
                         'linear_acceleration_covariance_1', 'linear_acceleration_covariance_2',
                         'linear_acceleration_covariance_3', 'linear_acceleration_covariance_4',
                         'linear_acceleration_covariance_5', 'linear_acceleration_covariance_6',
                         'linear_acceleration_covariance_7', 'linear_acceleration_covariance_8',
                         'child_frame_id', 'pose.pose.position.x',
                         'pose.pose.position.y', 'pose.pose.position.z',
                         'pose.pose.orientation.x', 'pose.pose.orientation.y',
                         'pose.pose.orientation.z', 'pose.pose.orientation.w', 'pose.covariance',
                         'twist.twist.linear.x', 'twist.twist.linear.y', 'twist.twist.linear.z',
                         'twist.twist.angular.x', 'twist.twist.angular.y',
                         'twist.twist.angular.z', 'twist.covariance',
                         'values']
    dataframe = dataframe[['Time',
                           'linear.x', 'angular.z', # cmd_vel
                           'linear_acceleration.x', 'linear_acceleration.y', 'linear_acceleration.z', 'angular_velocity.x', 'angular_velocity.y', 'angular_velocity.z', # imu
                           'pose.pose.position.x', 'pose.pose.position.y', 'twist.twist.linear.x', 'twist.twist.angular.z', # odom
                           'values']] # servo

    # trim first and last rows for more coherent data
    clip_var = int(len(dataframe) * .1)
    dataframe = dataframe.iloc[clip_var:-clip_var].reset_index(drop=True)

    # fill missing values
    dataframe.ffill(inplace=True)
    dataframe.bfill(inplace=True)

    # set initial timestep at 0
    dataframe['Time'] -= dataframe['Time'].min()

    # remove gravity from acceleration wrt z axis
    dataframe['linear_acceleration.z'] -= dataframe['linear_acceleration.z'].mean()
    
    # get power values scaled down by 1e3 factor
    data = dataframe['values'].tolist()
    data = [line.replace(', ', '];[') for line in data]
    data = [line.replace('[', '') for line in data]
    data = [line.replace(']', '') for line in data]
    data = [line.split(sep=';') for line in data]
    data = [[yaml.safe_load(line) for line in separate_lines] for separate_lines in data]
    columns = ['Power_' + str(idx + 1) for idx in range(len(data[0]))]
    columns_left = ['Power_' + str(idx + 1) for idx in range(len(data[0])) if idx % 2 == 0]
    columns_right = ['Power_' + str(idx + 1) for idx in range(len(data[0])) if idx % 2 != 0]
    power = pd.DataFrame([[abs(tuple(dictionary.values())[1] * tuple(dictionary.values())[2]) / 1e3 for dictionary in line] for line in data], columns=columns)
    power_left = pd.DataFrame([[abs(tuple(dictionary.values())[1] * tuple(dictionary.values())[2]) / 1e3 for idx, dictionary in enumerate(line) if idx % 2 == 0] for line in data], columns=columns_left)
    power_right = pd.DataFrame([[abs(tuple(dictionary.values())[1] * tuple(dictionary.values())[2]) / 1e3 for idx, dictionary in enumerate(line) if idx % 2 != 0] for line in data], columns=columns_right)
    dataframe = pd.concat([dataframe, power], axis=1)
    dataframe['mean_power'] = power.mean(axis=1)
    dataframe['mean_power_left'] = power_left.mean(axis=1)
    dataframe['mean_power_right'] = power_right.mean(axis=1)
    dataframe.drop(columns=['values'], inplace=True)

    # write dataframe to csv
    dataframe.to_csv(target_dir.joinpath(key + '.csv'))
    
    # gather labels
    sample_dict = {'surface': subfolders[3], 'kinematics': subfolders[4], 'spacing': subfolders[5], 'trajectory': subfolders[6]}
    labels[key] = sample_dict

# dump labels to json
with open('../data/labels.json', 'w') as fp:
    json.dump(labels, fp)

KeyboardInterrupt: 